# Julia *gotchas* and how to handle them
(Inspired by http://www.stochasticlifestyle.com/7-julia-gotchas-handle/ by Chris Rackauckas.)

**One can write terribly slow code in any language, including Julia.**

Below we address common performance *gotchas* in Julia code.

# Gotcha 1: Global scope

In [ ]:
a=2.0
b=3.0
function linearcombo()
  return 2a+b
end
answer = linearcombo()

@show answer;

The issue here is that the REPL/global scope does not guarantee that `a` and `b` are of a certain type.

### How to identify and avoid this issue?

One way to identify the issue is [Traceur.jl](https://github.com/MikeInnes/Traceur.jl). It is basically a codified version of the [performance tips](https://docs.julialang.org/en/v0.6.4/manual/performance-tips/#man-performance-tips-1) in the Julia documentation.

#### 1) Wrap code in functions.

In [ ]:
function outer()
    a=2.0; b=3.0
    function linearcombo()
      return 2a+b
    end
    return linearcombo() 
end

answer = outer()

@show answer;

This is fast.

In fact, it's not just fast, but as fast as it can be! Julia has figured out the result of the calculation at compile-time and returns **just the result (a literal)!**

(Effectively, `outer() = 7` at run-time.)

#### 2) Declare globals as (compile-time) constants.

In [ ]:
const A=2.0
const B=3.0

function Linearcombo()
  return 2A+B
end
answer = Linearcombo()

@show answer;

Note that the constants above are only compile-time constants, which can be modified:

#### Take home message: Always wrap a performance critical piece of code in a self-contained function.

# Gotcha 2: Type-instabilities

What's bad for performance about the following function?

In [ ]:
function g()
  x=1
  for i = 1:10
    x = x/2
  end
  return x
end

A more drastic example

### How to find and deal with type-instabilities

#### 1) Avoid type changes

Initialize `x` as `Float64` and it's fast.

In [ ]:
function h()
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

#### 2) Detect issues with `@code_warntype` (or `@trace`)

(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by *union splitting*. See the blog post by Tim Holy: https://julialang.org/blog/2018/08/union-splitting)

#### 3) The C/Fortran way: specify types (to get errors or to heal the problem by conversion)

In [ ]:
function g2()
  x::Int64 = 1
  for i = 1:10
    x = x/2
  end
  return x
end

In [ ]:
function g3()
  x::Float64 = 1 # triggers an implicit conversion to Float64
  for i = 1:10
    x = x/2
  end
  return x
end

#### 4) Function barriers

In [ ]:
arr = Vector{Union{Int64,Float64}}(undef, 4)
arr[1]=4
arr[2]=2.0
arr[3]=3.2
arr[4]=1
arr

In [ ]:
function foo(array)
  for i in eachindex(array)
    val = array[i]
    val^2
  end
end

In [ ]:
function inner_foo(val)
  # Do algorithm X on val
  val^2
end
 
function foo2(array)
  for i in eachindex(array)
    inner_foo(array[i])
  end
end

#### Comments:

Why Allow Type-Instabilities in the first place? Convenience vs performance tradeoff.

Note that type instabilities can naturally occur (reading files, user input etc.) so not any red marker is bad/avoidable.

Note that Julia is smart and a changing type isn't *per se* an issue:

In [ ]:
function g4()
  x=1
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

#### Take home message: watch out for type-instabilities in performance critical parts of your code.

# Gotcha 3: Views and copies

# Gotcha 4: Temporary allocations and vectorized code

In [ ]:
using BenchmarkTools

In [ ]:
function f()
  x = [1;5;6]
  for i in 1:100_000
    x = x + 2*x
  end
  return x
end

In [ ]:
@btime f();

### How to handle it? -> More dots or more explicity

Great blog post by Steven G. Johnson: https://julialang.org/blog/2017/01/moredots ([related notebook](https://github.com/JuliaLang/www.julialang.org/blob/master/blog/_posts/moredots/More-Dots.ipynb))

In [ ]:
function f()
    x = [1;5;6]
    for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end
@btime f();

In [ ]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        x = x .+ 2 .* x
    end
    return x
end
@btime f();

In [ ]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        x .= x .+ 2 .* x
    end
    return x
end
@btime f();

In [ ]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        @. x = x + 2*x
        # or @. x = x + 2*x
    end
    return x
end
@btime f();

### Extra trick: `@inbounds`

In [ ]:
function f()
    x = [1;5;6]
    @inbounds for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2*x[k]
        end
    end
    return x
end
@btime f();

# Gotcha 5: Abstract fields

In [ ]:
using BenchmarkTools

In [ ]:
struct MyType
    x::AbstractFloat
    y::AbstractString
end

f(a::MyType) = a.x^2 + sqrt(a.x)

In [ ]:
a = MyType(3.0, "test")

@btime f($a);

In [ ]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

In [ ]:
b = MyTypeConcrete(3.0, "test")

@btime f($b);

Note that the latter implementation is **more than 30x faster**!

### How to handle it?

But what if I want to accept any kind of `AbstractFloat` and `AbstractString` in my type?

Use type parameters!

In [ ]:
struct MyTypeParametric{A<:AbstractFloat, B<:AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

In [ ]:
c = MyTypeParametric(3.0, "test")

From the type alone the compiler knows what the structure contains and can produce optimal code:

In [ ]:
@btime f($c);

In [ ]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

In [ ]:
@btime f($c);

# Gotcha 6: Writing to global scope

In [ ]:
# Try this in the Julia REPL
a = 0
for i in 1:10
    a += i
end

(For more information, see the "official" discussion here: https://github.com/JuliaLang/julia/issues/28789)

#### Take home message: again, just wrap things into functions.

# Gotcha 7: Column major order

In [ ]:
M = rand(1000,1000);

function fcol(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

In [ ]:
@btime fcol($M)

In [ ]:
@btime frow($M)

#### Take home message: fastest varying index goes first!

# Gotcha 8: Lazy operations

Let's say we want to calculate `X = M + (M' + 2*I)`.

In [ ]:
using LinearAlgebra

In [ ]:
M = [1 2; 3 4]
M + (M' + 2*I)

Now let's assume that, for some reason, we want to implement it more explicitly. Something along the lines of

In [ ]:
function calc(M)
    X = M'
    X[1,1] += 2
    X[2,2] += 2
    M + X
end

Let's check for correctness.

In [ ]:
calc([1 2; 3 4]) == M + (M' + 2*I)

Somehow it's not correct!

### How to solve this?

The "issue" is that `M'` makes a lazy adjoint of `M`. It is just another way of looking at the same piece of memory. Hence, when we do `X[1,1] += 1` we are actually changing `M`, leading to a wrong result. We can heal this by enforcing a `copy`:

In [ ]:
function calc_corrected(M)
    X = copy(M')
    X[diagind(X)] .+= 2
    M + X
end

In [ ]:
calc_corrected([1 2; 3 4]) == M + (M' + 2*I)

This isn't really an issue. In fact, this lazyness (+ allocation free identity matrix) is precisley the reason why the straightforward solution is fast!

In [ ]:
function calc_straightforward(A)
    A + (A' + 2*I)
end

@btime calc($[1 2; 3 4]);
@btime calc_corrected($[1 2; 3 4]);
@btime calc_straightforward($[1 2; 3 4]);

### Extra tip: Comprehensions and generators

The construct is known as a [comprehension](https://docs.julialang.org/en/v1/manual/arrays/#Comprehensions-1).

To avoid the temporary array that the comprehension creates, we can also write the comprehension withouth square brackets. This creates a so-called [generator expression](https://docs.julialang.org/en/v1/manual/arrays/#Generator-Expressions-1).

# Core messages of this Notebook

* Gotcha 1: **Wrap code in self-contained functions** in performance critical applications, i.e. avoid global scope.
* Gotcha 2: Write **type-stable code** (check with `@code_warntype`).
* Gotcha 3: Use **views** instead of copies to avoid unnecessary allocations.
* Gotcha 4: Use **broadcasting (more dots)** to avoid temporary allocations in vectorized code (or write out loops).
* Gotcha 5: **Types should always have concrete fields.** If you don't know them in advance, use type parameters.


* Gotcha 6: Be aware of the **scoping rules** in non-Jupyter-notebook environments.
* Gotcha 7: Be aware of **column major order** when looping over arrays.
* Gotcha 8: Be aware of **lazy operations** like, for example, transpose.